# MIP für makespan

*mixed integer program* für das minimum makespan-Problem:

Gegeben eine Menge $J$ von Aufträgen, die auf einer Menge $M$ von Maschinen bearbeitet werden können, wobei $d_{ap}$ die Bearbeitungsdauer von Auftrag $a$ auf Maschine $p$  bezeichnet, finde eine Zuteilung $f:J\to M$, die den makespan minimiert. Der makespan ist dabei

$$
\text{make}(f)= \max_{p\in M} \sum_{a\in J:f(p)=a} d_{ap}
$$

Als mip:

$$
\begin{array}{lll}
\min & T & \\
\text{unter }& 
\sum_{a\in J} x_{ap}d_{ap} \leq T&\text{ für alle }p\in M\\
&\sum_{p\in M} x_{ap} = 1 & \text{ für alle }a\in J\\
&x_{ap}\in\{0,1\} &\text{ für alle } a\in J\text{ und }p\in M
\end{array}
$$

Das Paket mip muss installiert sein. 

In [ ]:
# Zum Installieren in google colab, entfernen Sie die Raute in der nächsten Zeile
#!pip install mip

In [1]:
import mip
import random  ## Standard-Bibliothek zur Erzeugung von Zufallszahlen

Als erstes definieren wir eine Methode zur Erzeugung von Zufallsinstanzen. Die Funktion <code>random.randint(a,b)</code> liefert eine zufällig ganze Zahl zwischen a und b (einschließlich).

In [2]:
def random_instance(number_machines,number_jobs):
    d=[[random.randint(1,20) for _ in range(number_machines)] for _ in range(number_jobs)]
    return d

In [3]:
number_jobs=10
number_machines=3
D=random_instance(number_machines,number_jobs)
D

[[9, 5, 9],
 [16, 5, 20],
 [19, 17, 15],
 [8, 10, 15],
 [8, 4, 15],
 [6, 15, 18],
 [3, 12, 3],
 [14, 1, 17],
 [8, 7, 15],
 [19, 18, 18]]

Wir stellen das mip auf. Dazu definieren wir zunächst das mip-Objekt und fügen gleich Variablen hinzu, einmal die Variablen $x_{ap}\in\{0,1\}$ für Auftrag $a$ und Maschine $p$ und dann noch die makespan-Variable $T$, die alle (nicht-negativen) Werte annehmen darf. Durch Angabe von <code>var_type=mip.BINARY</code> definieren wir Binärvariablen. Mit <code>var_type=INTEGER</code> würden wir allgemeine Ganzzahlvariablen definieren.

In [4]:
m=mip.Model()
x=[[m.add_var(var_type=mip.BINARY) for _ in range(number_machines)] for _ in range(number_jobs)]
T=m.add_var()

Wir fügen die Bedingung

$\sum_{a\in J}x_{ap}d_{ap}\leq T\quad\text{ für alle }p\in M$

hinzu.

In [5]:
for p in range(number_machines):
    m+=mip.xsum(x[a][p]*D[a][p] for a in range(number_jobs))<=T

Schließlich müssen wir garantieren, dass auch jeder Auftrag bearbeitet wird:

$\sum_{p\in M}x_{ap}=1\quad\text{für alle }a\in J$

In [6]:
for a in range(number_jobs):
    m+=mip.xsum(x[a][p] for p in range(number_machines))==1

Wir starten die Optimierung:

In [7]:
m.objective=mip.minimize(T)
m.optimize()    

<OptimizationStatus.OPTIMAL: 0>

Schließlich ist's zweckmäßig sich die Lösung auszugeben:

In [13]:
for p in range(number_machines):
    print("Maschine {}:".format(p))
    jobs=[a for a in range(number_jobs) if x[a][p].x==1]
    print("  Aufträge: {}".format(jobs))
    job_lengths=[D[a][p] for a in jobs]
    print("  Auftragsdauern: {}".format(job_lengths))
    print("  Arbeitsdauer Maschine {}: {}".format(p,sum(job_lengths)))
print("-------")
print("makespan: {}".format(T.x))

Maschine 0:
  Aufträge: [3, 5, 6, 8]
  Auftragsdauern: [8, 6, 3, 8]
  Arbeitsdauer Maschine 0: 25
Maschine 1:
  Aufträge: [1, 2, 4, 7]
  Auftragsdauern: [5, 17, 4, 1]
  Arbeitsdauer Maschine 1: 27
Maschine 2:
  Aufträge: [0, 9]
  Auftragsdauern: [9, 18]
  Arbeitsdauer Maschine 2: 27
-------
makespan: 27.0
